In [48]:
import dicom
import os.path
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import math
from random import shuffle
#MAIN_FOLDER = "sample_images"
MAIN_FOLDER = "stage1/stage1"
SCALE_CONST = 16
hyper_param = 0.001

def compute_score(labels,pred):
	error_count = 0
	for tup in zip(pred,labels):
		if tup[0]!=tup[1]:
			error_count+=1
	return float(labels.shape[0]-error_count)/float(labels.shape[0])

def LDA_predict(cov,x_list,pi_list,mean_list):
    inv_cov = np.linalg.pinv(cov+hyper_param*np.identity(cov.shape[0]))
    log_det = sum([math.log(eig+hyper_param) for eig in np.linalg.eigvals(cov)])
    pred = []
    for x in x_list:
        estimated_g = []
        for i in range(len(mean_list)):
            estimated_g.append(-0.5*np.dot(np.dot((x-mean_list[i]).T,inv_cov),(x-mean_list[i]))+math.log(pi_list[i])-0.5*log_det)
            #estimated_g.append(np.dot(np.dot(x.T,inv_cov),mean_list[i])-0.5*np.dot(np.dot(mean_list[i].T,inv_cov),mean_list[i])+math.log(pi_list[i]))
        pred.append(np.argmax(estimated_g))
    return pred

def QDA_predict(cov_list,x_list,pi_list,mean_list):
    pred = []
    inv_cov_list = [np.linalg.pinv(cov_list[i]+hyper_param*np.identity(cov_list[i].shape[0])) for i in range(len(mean_list))]
    eigvals = np.linalg.eigvals(cov_list[i])
    log_det_list = [sum([math.log(eig+hyper_param) for eig in eigvals]) for i in range(len(mean_list))]
    
    for x in x_list:
        estimated_g = []
        for i in range(len(mean_list)):
            inv_cov = inv_cov_list[i]
            estimated_g.append(-0.5*np.dot(np.dot((x-mean_list[i]).T,inv_cov),(x-mean_list[i]))+math.log(pi_list[i])-0.5*(log_det_list[i]))
        pred.append(np.argmax(estimated_g))
    return pred
#print patient_id_list
#df_labels = pd.read_csv("stage1_labels.csv")
#df_labels = df_labels.drop('index')
#print df_labels[(df_labels['id']=='0015ceb851d7251b8f399e39779d1e7d')].loc[:,'cancer']
patient_id_list = [name for name in os.listdir(MAIN_FOLDER)]
shuffle(patient_id_list)
label_list = {}
with open("stage1_labels.csv") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        if row[1]=='0' or row[1]=='1':
            label_list[row[0]]=int(row[1])
print "point1"
test_set = []
with open("stage1_sample_submission.csv") as csvfile:
    reader = csv.reader(csvfile,delimiter=',')
    for row in reader:
        if row[0]!='id':
            test_set.append(row[0])
print "point2"
one_counter = 0
one_patient_list = []
zero_counter = 0
zero_patient_list = []
for key,value in label_list.items():
    if value==0:
        zero_counter+=1
        zero_patient_list.append(key)
    elif value==1:
        one_counter+=1
        one_patient_list.append(key)
    else:
        print "WTF"
print "zerocounter:",zero_counter
print "onecounter:",one_counter
num_samples = 100
sample_patient_id_list = one_patient_list[:num_samples//2]+zero_patient_list[:num_samples//2]

point1
point2
zerocounter: 1035
onecounter: 362


In [49]:
mid_images = []
counter=0
for patient_id in sample_patient_id_list:
#for patient_id in patient_id_list:
#for i in range(100):
    if patient_id not in test_set:
    #if patient_id_list[i] not in test_set:
        files = [f for f in os.listdir(MAIN_FOLDER+"/"+patient_id) if os.path.isfile(os.path.join(MAIN_FOLDER+"/"+patient_id, f))]
        #files = [f for f in os.listdir(MAIN_FOLDER+"/"+patient_id_list[i]) if os.path.isfile(os.path.join(MAIN_FOLDER+"/"+patient_id_list[i], f))]
        num_files = len(files)
        image = dicom.read_file(MAIN_FOLDER+"/"+patient_id+"/"+files[num_files//2])
        #image = dicom.read_file(MAIN_FOLDER+"/"+patient_id_list[i]+"/"+files[num_files//2])
        image_features = [np.mean([image.pixel_array[SCALE_CONST*i+i2][SCALE_CONST*j+j2] for j2 in range(SCALE_CONST) for i2 in range(SCALE_CONST)]) for j in range(image.Columns/SCALE_CONST) for i in range(image.Rows/SCALE_CONST)]
        image_features.append(label_list[patient_id]) #last column is of labels
        print "current patient label",label_list[patient_id]
        #print "current patient label",label_list[patient_id_list[i]]
        #image_features.append(label_list[patient_id_list[i]])
        mid_images.append(image_features)
        counter+=1
        print "num patients done:",counter
print "point3"

KeyboardInterrupt: 

In [50]:
shuffle(mid_images)
for image in mid_images:
    print "image label:",image[len(image)-1]
validation_set = np.array(mid_images[:len(mid_images)//5])
training_set = np.array(mid_images[len(mid_images)//5:])
#print "validationsetshape",validation_set.shape
#print "trainingsetshape",training_set.shape
LDA_errors = []
QDA_errors = []
classes = []
#print "traininglabels",training_set[:,training_set.shape[1]-1]
#print "validationlabels",validation_set[:,validation_set.shape[1]-1]
# for label in training_set[:,training_set.shape[1]-1]:
#     print label
#     if label not in classes:
#         classes.append(label)
print "point4"
nums_samples = [counter]
num_classes = 2
#nums_samples = [len(mid_images)]
#nums_samples = [100,200,500,1000,len(mid_images)]
for num_samples in nums_samples:
    v_set = validation_set[:,:validation_set.shape[1]-1]
    v_label = validation_set[:,validation_set.shape[1]-1]
    t_set = training_set[:num_samples,:training_set.shape[1]-1]
    t_label = training_set[:num_samples,training_set.shape[1]-1]
    #print "t_label", t_label
    #print "vlabel:",v_label
    classes = [[] for _ in range(num_classes)]
    classes_means = []
    classes_covs = []
    for i in range(len(t_label)):
        classes[int(t_label[i])].append(t_set[i])
    print "point5"
    for class_label in classes:
        class_label = np.array(class_label)
        classes_means.append(np.mean(class_label,axis=0))
        classes_covs.append(np.cov(class_label.T))
    print "point6"
    mean_cov = np.array([[0 for _ in range(t_set.shape[1])] for _ in range(t_set.shape[1])])
    for cov in classes_covs:
        mean_cov = mean_cov+cov
    mean_cov = mean_cov/float(num_classes)
    pi_list = [float(len(class_label))/float(t_set.shape[0]) for class_label in classes]
    LDA_pred = LDA_predict(mean_cov,v_set,pi_list,classes_means)
    print "point7"
    QDA_pred = QDA_predict(classes_covs,v_set,pi_list,classes_means)
    print "point8"
    LDA_score = compute_score(v_label,LDA_pred)
    QDA_score = compute_score(v_label,QDA_pred)
    LDA_errors.append(1.0-LDA_score)
    QDA_errors.append(1.0-QDA_score)
    print "point9"
plt.plot(nums_samples,LDA_errors,'r-',label='LDA error')
plt.plot(nums_samples,QDA_errors,'b-',label='QDA error')
plt.title('Num Samples vs. Error Rate')
plt.xlabel('Number of Samples')
plt.ylabel('Error Rate')
plt.show()
print "LDA errors:",LDA_errors
print "QDA_errors:",QDA_errors

image label: 1
image label: 1
image label: 1
image label: 1
point4


IndexError: tuple index out of range